In [147]:
import requests
import re
import pandas as pd

In [ ]:
headers = {
    'Accept-Language': 'zh-CN,zh;q=0.9'
}

In [78]:
def parse_table(url):
    # url = 'https://zh.wikipedia.org/wiki/Wikipedia:%E7%BB%9F%E8%AE%A1'
    # url = 'https://zh.wikipedia.org/wiki/Help:%E7%9B%AE%E5%BD%95'
    
    # 发送请求获取 HTML 内容
    response = requests.get(url, headers=headers)
    
    # 使用 pandas.read_html 解析 HTML 内容并获取表格数据
    tables = pd.read_html(response.content)

    return tables

In [126]:
url = 'https://zh.wikipedia.org/wiki/Wikipedia:%E7%BB%9F%E8%AE%A1'
tables = parse_table(url)

In [1]:
def process_table(table):
    
    del_list = ['机器人', '搜索引擎', '访问者来源', '编辑者来源', '全部空间']

    
    # 表格需要多于2行, 但是不能多于500行
    if len(table) < 2  or len(table) > 200:
        return

    col_names = table.columns.tolist()
    # 表格不能小于2列
    if len(col_names) < 2 or len(col_names) > 10:
        return
        
    # 表格如果没有真实含义的header 则删除
    if col_names[:2] == [0, 1]:
        return
    
    # 如果有多层Header（多层Header可能来源于合并表格），那么转换成新的header
    if isinstance(col_names[0], tuple):  # 如果多层，会以truple的形式
        if len(col_names[0]) > 3 :
            return
            
        elif len(col_names[0]) == 2:  # 合并table的header
            col_names = ['_'.join(item) if item[0] != item[1] else item[0]  for item in table.columns.tolist()]
            table.columns = col_names
            
        elif len(col_names[0]) == 3:  # 对于三行的，直接组合
            col_names = ['_'.join(item) for item in table.columns.tolist()]
            table.columns = col_names
        
    # 如果列名的长度总和大于100，则删除
    if len(''.join(col_names)) > 100:
        return

    # 如果在删除字段表里则删除
    if any(re.search(keyword, ''.join(col_names)) for keyword in del_list):
        return
        

    return table
    



In [163]:
# 对获取某个页面上的所有表格执行如下筛选

for table in tables:

    table = process_table(table)    
    if table is not None:  # 如果table 不为空
        print(table)
    

  Unnamed: 0          条目          总页数           编辑数   管理员           用户   
0         统计  ▲1,410,133  ▲ 7,656,650  ▲ 81,642,461  ▼ 63  ▲ 3,486,419  \
1         排名        ▲ 12         ━ 10           ━ 8   ▲ 7          ━ 6   

      活跃用户        文件   条目深度  
0  ▼ 7,616  ▲ 66,397  ▲ 209  
1      ▼ 9      ━ 11   ▲ 25  
          条目数           日期                      成长
0    隐藏显示过去数据     隐藏显示过去数据                隐藏显示过去数据
1           1  2002年10月24日                 主页建成[1]
2        1000   2003年7月12日    285天增加1000条，平均1天增加4条
3        2000   2003年9月21日    71天增加1000条，平均1天增加14条
4        3000  2003年11月21日    61天增加1000条，平均1天增加16条
..        ...          ...                     ...
195   1360000   2023年6月10日  39天增加10000条，平均1天增加256条
196   1370000   2023年7月31日  51天增加10000条，平均1天增加196条
197   1380000   2023年10月5日  66天增加10000条，平均1天增加152条
198   1390000   2023年12月1日  57天增加10000条，平均1天增加175条
199   1400000   2024年1月21日  51天增加10000条，平均1天增加196条

[200 rows x 3 columns]
            日期  埃及阿拉伯语版 埃及阿拉伯语版.1      差距  差距.1    

In [77]:
tables[13]

,编辑者来源（2021年7月）[2],编辑者来源（2021年7月）[2].1,编辑者来源（2021年7月）[2].2,编辑者来源（2021年7月）[2].3,编辑者来源（2021年7月）[2].4
0,国家或地区,国家或地区,NaN,百分比,百分比
1,台湾,台湾,NaN,46.2%,46.2%
2,香港,香港,NaN,30.8%,30.8%
3,美国,美国,NaN,7.2%,7.2%
4,马来西亚,马来西亚,NaN,4.6%,4.6%
5,日本,日本,NaN,2.9%,2.9%
6,加拿大,加拿大,NaN,1.6%,1.6%
7,澳门,澳门,NaN,1.5%,1.5%
8,英国,英国,NaN,1.5%,1.5%
9,德国,德国,NaN,0.9%,0.9%


In [15]:
table.columns.tolist() == [0, ]

False

['语言', '条目总数', '典范条目_数目', '典范条目_比例', '优良条目_数目', '优良条目_比例']

'a_a'